# Import libraries

In [1]:
import pandas as pd
import sys
import numpy as np
import random

# Import data

In [191]:
df1 = pd.read_excel('Dane_S2_50_10.xlsx')
df1 = df1.drop('Zadanie',1)

df2 = pd.read_excel('Dane_S2_100_20.xlsx')
df2 = df2.drop('Zadanie',1)

df3 = pd.read_excel('Dane_S2_200_20.xlsx')
df3 = df3.drop('Zadanie',1)

# Functions

In [302]:
def CountScore(df):
    m_czasy = []
    v_czasy = []

    for i in range(len(df)):
        m_czasy.append([0] * len(df.columns))

    for i in range(len(df)):
        for j in range(len(df.columns)):
            if(i!=0 and j!=0):
                czas = max( m_czasy[i-1][j] + df.iloc[i,j],m_czasy[i][j-1] + df.iloc[i,j]) 
                m_czasy[i][j] = czas
            elif(i==0 and j==0):
                czas = df.iloc[i,j]
                m_czasy[i][j] = czas
            elif(i==0):
                czas = max(df.iloc[i,j],m_czasy[i][j-1] + df.iloc[i,j])
                m_czasy[i][j] = czas
            elif(j==0):
                czas = max(df.iloc[i,j],m_czasy[i-1][j] + df.iloc[i,j])
                m_czasy[i][j] = czas

    return m_czasy[i][j]

def Cross(assigment1,assigment2,idx=5):
    idx=random.randrange(0,len(assigment1),1)
    #print(idx)

    a=assigment1[:idx], assigment1[idx:]
    b=assigment2[:idx], assigment2[idx:]
    
    child1_part1 = a[0]
    child1_part2 = a[1]
    child2_part1 = b[0]
    child2_part2 = b[1]

    child1 = child1_part1 + [ x for x in child2_part2 if not x in child1_part1]
    child1 = child1 + [ x for x in child1_part2 if not x in child1]
    
    child2 = child2_part1 + [ x for x in child1_part2 if not x in child2_part1]
    child2 = child2 + [ x for x in child1_part1 if not x in child2]
    
    return child1, child2
  
def CrossDf(df1,df2,idx=5):
    idx=random.randrange(0,len(df1),1)
    #print(idx)
    
    a=df1[:idx], df1[idx:]
    b=df2[:idx], df2[idx:]

    child1_part1 = list(a[0].index) 
    child1_part2 = list (a[1].index)
    
    child2_part1= list(b[0].index) 
    child2_part2 = list(b[1].index) 
    
    child1 = child1_part1 + [ x for x in child2_part2 if not x in child1_part1]
    child1 = child1 + [ x for x in child1_part2 if not x in child1]
    
    child2 = child2_part1 + [ x for x in child1_part2 if not x in child2_part1]
    child2 = child2 + [ x for x in child1_part1 if not x in child2]
    
    return df1.iloc[child1], df1.iloc[child2]

# def CrossAlt():

# def Mutation():
    #simpe swap
    
    
# def MutationAlt():
    #inwersja

def Ranking(temp,n_top=50):
    temp = temp.sort_values('score')
    temp = temp[:n_top] #bierzemy top 50 najlpszych
    temp = temp.reset_index(drop=True) 
    return temp
    
def Tournament(temp):
    winners = []
    temp_bc = temp
    while(len(temp)>0):
        knights = list(temp.sample(n=2).index)
        if(temp.loc[knights[0]].score > temp.loc[knights[1]].score):
            winners.append(knights[0])
        else:
            winners.append(knights[1])
        temp = temp.drop(knights)
        
    temp = temp_bc.loc[winners].reset_index(drop=True)
    return temp


def shuffleACopy(x):
        b = x[:] # make a copy of the keys
        random.shuffle(b) # shuffle the copy
        return b # return the copy

# Main loop

## Data 50x10

In [349]:
%%time

n_pop = 500 #liczba osobnikow w populacji
n_top = 50
n_generations = 20
population = [shuffleACopy(list(df.index)) for x in range(n_pop)] 

generation_records_mean = []

for generation in range(n_generations):
    temp = pd.DataFrame(columns=['score','assignment'])

    for i,pop in enumerate(population):
        score = CountScore(df.iloc[pop])
        temp.loc[i] = [score, pop]

    best_score = temp.sort_values('score')[:1]
        
###   METODA RANKINGU ###
#     1stx2nd, 3rdx4th, 5thx6th itd. --> malo efektywna
    temp = Ranking(temp,n_top)

###    METODA TURNIEJU ###
#     temp = Tournament(temp)
#     wyniki wychodza coraz gorsze w tej metodzie(?)
    
    r1 = temp[temp.index%2 == 0] #rodzic1
    r2 = temp[temp.index%2 == 1].reset_index(drop=True) #rodzic2
       
    new_pop =[]
    for i,rs1 in enumerate(r1.assignment):
        children = Cross(rs1, r2.assignment[i])
        new_pop.append(children[0])
        new_pop.append(children[1])

    
    population = temp.assignment.tolist() + new_pop
    if( int(temp.sort_values('score')[:1].score) > int(best_score.score) ):
        best_score = temp.sort_values('score')[:1]
    generation_records_mean.append(temp.score.mean())

best_score

CPU times: user 23.8 s, sys: 63.3 ms, total: 23.8 s
Wall time: 24.1 s


,score,assignment
0,3376,"[5, 28, 0, 48, 10, 7, 43, 13, 24, 33, 41, 23, ..."


In [352]:
generation_records

[3658.14,
 3638.24,
 3629.7,
 3617.54,
 3602.7,
 3592.82,
 3585.32,
 3574.72,
 3566.02,
 3552.96,
 3541.08,
 3526.18,
 3509.8,
 3504.92,
 3504.92,
 3504.84,
 3504.68,
 3504.36,
 3503.72,
 3502.44]

In [350]:
#run to save best score 
# current: 3376
best_score.to_csv(path_or_buf='best.csv')

In [182]:
# %%time

# knights = list(temp.sample(n=2).index)
# print(knights[0])
# temp.loc[knights[0]].score

63


4036

# Notes

In [5]:
# abc = [11,12,13]
# abc + abc
# len([5, 1, 2]+[1,2])
# [ x for x in list(Cross(df,df,5)[1].index) if not x in abc]
# Cross(df2, df2.sample(n=len(df2)), 5)[0]
# df1.sample(n=len(df1))
# Cross(df, df.sample(n=len(df)), 5)[0]

# zzz=list(df.index)
# random.shuffle(zzz)
# Cross(list(df.index),zzz)[0]